In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.11.4


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [13]:
!ls ../../..

README.MD      images         mlruns         poetry.lock
data           mlflow.db      models         pyproject.toml
homework       mlops          notebooks


In [14]:
df = read_data('../../../data/yellow_tripdata_2023-03.parquet')


In [15]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [18]:
print('y_pred: ', pd.DataFrame(y_pred).describe())

y_pred:                    0
count  3.316216e+06
mean   1.420387e+01
std    6.247490e+00
min   -1.160751e+01
25%    1.096814e+01
50%    1.197354e+01
75%    1.413772e+01
max    7.498748e+01


In [20]:
df_result=pd.DataFrame(y_pred)
df_result['ride_id'] = f'{2023:04d}/{3:02d}_' + df.index.astype('str')

In [21]:

df_result.to_parquet(
    'prediction_output',
    engine='pyarrow',
    compression=None,
    index=False
)

In [22]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1335 bytes to starter.py
